In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle

In [0]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/auto.csv')

In [0]:
df=df.replace(-1,np.nan)

In [0]:
def handleMIssingVals(df2):
  df2['ps_ind_02_cat'] = df2['ps_ind_02_cat'].fillna(1.0)
  df2['ps_ind_04_cat']=df2['ps_ind_04_cat'].fillna(0.0)
  df2['ps_ind_05_cat']=df2['ps_ind_05_cat'].fillna(7.0)
  df2['ps_reg_03']=df2['ps_reg_03'].fillna(df2['ps_reg_03'].mean())
  df2['ps_car_01_cat']=df2['ps_car_01_cat'].fillna(11.0)
  df2['ps_car_05_cat']=df2['ps_car_05_cat'].fillna(2.0)
  df2['ps_car_07_cat']=df2['ps_car_07_cat'].fillna(1.0)
  df2['ps_car_09_cat']=df2['ps_car_09_cat'].fillna(2.0)
  df2['ps_car_11']=df2['ps_car_11'].fillna(3.0)
  df2['ps_car_12']=df2['ps_car_12'].fillna(df2['ps_car_12'].mean())
  df2['ps_car_14']=df2['ps_car_14'].fillna(df2['ps_car_14'].mean())

In [0]:
handleMIssingVals(df)

In [0]:
target=df['target']

In [0]:
drop_cols=['id', 'ps_car_13' ,'ps_car_14' , 'ps_reg_03' , 'ps_ind_10_bin' , 'ps_ind_11_bin' , 'ps_ind_13_bin' ,'ps_ind_14' , 'ps_car_03_cat','target']

In [0]:
df.drop(drop_cols,axis=1,inplace=True)

In [0]:
df.shape

(595212, 49)

In [0]:
rescale_cols=['ps_ind_01' ,'ps_ind_03','ps_ind_15','ps_car_11', 'ps_car_15', 'ps_calc_04', 'ps_calc_05','ps_calc_06','ps_calc_07','ps_calc_08','ps_calc_09','ps_calc_10','ps_calc_11','ps_calc_12','ps_calc_13','ps_calc_14']
scaler=StandardScaler()
scaled_Data=scaler.fit_transform(df[rescale_cols])

In [0]:
scaled_Data.shape

(595212, 16)

In [0]:
df_scaled=pd.DataFrame(scaled_Data,columns=rescale_cols)

In [0]:
df_scaled.shape

(595212, 16)

In [0]:
df.drop(rescale_cols,axis=1,inplace=True)

In [0]:
df=pd.concat([df,df_scaled],axis=1)

In [0]:
cat_cols=df[['ps_ind_02_cat','ps_ind_04_cat','ps_ind_05_cat','ps_car_01_cat','ps_car_04_cat','ps_car_05_cat','ps_car_06_cat','ps_car_10_cat']]

In [0]:
for eachcol in cat_cols:
  dummies=pd.get_dummies(cat_cols[eachcol],prefix_sep='_',prefix=eachcol)
  df=df.join(dummies)
data_vars=df.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_cols]
df_final=df[to_keep]

In [0]:
df=df_final

In [0]:
len(df.columns)

101

In [0]:
X_train,X_test,Y_train,Y_test=train_test_split(df,target,test_size=0.30,random_state=101)

In [0]:
pca = PCA(n_components=10)
#principalComponents = pca.fit_transform(df)

In [0]:
train_pca=pca.fit(X_train)

In [0]:
pca.n_components_

10

In [0]:
pca.explained_variance_

array([1.09004284e+03, 1.47130075e+00, 1.22521596e+00, 1.15666483e+00,
       1.01079170e+00, 1.00622713e+00, 1.00456796e+00, 1.00302770e+00,
       1.00166828e+00, 9.99537437e-01])

In [0]:
train_dta=pca.transform(X_train)
test_dta=pca.transform(X_test)

In [0]:
train_dta.shape

(416648, 10)

In [0]:
principalDF=pd.DataFrame(train_dta,columns=['Principal Component1','Principal Component2','Principal Component3','Principal Component4','Principal Component5','Principal Component6','Principal Component7','Principal Component8','Principal Component9','Principal Component10'])

In [0]:
finalDF=pd.concat([principalDF,target],axis=1)

In [0]:
finalDF.drop('target',axis=1,inplace=True)

In [0]:
lgModel=LogisticRegression()

In [0]:
lgModel.fit(train_dta,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
filename = '/content/drive/My Drive/Colab Notebooks/Logisticmodel.sav'
pickle.dump(lgModel, open(filename, 'wb'))

In [0]:
type(train_dta),type(test_dta)

In [0]:
test_dta.shape

In [0]:
pred_y=lgModel.predict(test_dta)

In [0]:
type(pred_y.shape),type(Y_test.shape)

(tuple, tuple)

In [0]:
lgModel.score(test_dta,Y_test)

0.963525682668399

In [0]:
from sklearn.metrics import confusion_matrix,classification_report

In [0]:
print(confusion_matrix(Y_test,pred_y))

[[172051      0]
 [  6513      0]]


In [0]:
print(classification_report(Y_test,pred_y))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98    172051
           1       0.00      0.00      0.00      6513

    accuracy                           0.96    178564
   macro avg       0.48      0.50      0.49    178564
weighted avg       0.93      0.96      0.95    178564



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.metrics import f1_score,recall_score,precision_score

In [0]:
print(f1_score(Y_test, pred_y, average='micro'))
print(f1_score(Y_test, pred_y, average='macro'))
print(f1_score(Y_test, pred_y, average='weighted'))

0.963525682668399
0.49071203456783086
0.9456272962011365


In [0]:
f1_score(Y_test,pred_y)

0.0

In [0]:
recall_score(Y_test,pred_y)

0.0

In [0]:
pred_score=precision_score(Y_test,pred_y)
pred_score

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [0]:
probs=lgModel.predict_proba(test_dta)

In [0]:
class_1=probs[:,1]


In [0]:
lgModel_class1=class_1 > 0.2
lgModel_class1 = lgModel_class1 * 1

In [0]:
np.unique(lgModel_class1,return_counts=True)

(array([0]), array([178564]))

In [0]:
class_0=probs[:,0]

In [0]:
lgModel_class0=class_1 > 0.2
lgModel_class0 = lgModel_class1 * 0
np.unique(lgModel_class0,return_counts=True)

(array([0]), array([178564]))